In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import os
import tools

sc.logging.print_header()

os.chdir(os.path.expanduser('/home/jovyan/Zach/scanpy/'))
adata = sc.read_h5ad('h5ad/meninges_integrated.h5ad')
adata

scanpy==1.8.2 anndata==0.7.6 umap==0.5.1 numpy==1.21.6 scipy==1.5.2 pandas==1.3.5 scikit-learn==0.23.2 statsmodels==0.12.1 python-igraph==0.8.3 leidenalg==0.8.2 pynndescent==0.5.2


AnnData object with n_obs × n_vars = 10839 × 3266
    obs: 'sample_id', 'condition', 'site', 'sequencing_batch', 'scrublet_score', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'gmm_pct_count_clusters_keep', 'is_doublet', 'filter_rna', 'S_score', 'G2M_score', 'phase', 'leiden', 'GSE23925_DARK_ZONE_VS_NAIVE_BCELL_DN', 'GSE23925_DARK_ZONE_VS_NAIVE_BCELL_UP', 'GSE23925_LIGHT_ZONE_VS_DARK_ZONE_BCELL_DN', 'GSE23925_LIGHT_ZONE_VS_DARK_ZONE_BCELL_UP', 'GSE23925_LIGHT_ZONE_VS_NAIVE_BCELL_DN', 'GSE23925_LIGHT_ZONE_VS_NAIVE_BCELL_UP', 'leiden_R', 'celltype', 'keep'
    var: 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'celltype_colors', 'condition_colors', 'filter_rna_colors', 'hvg', 'is_doublet_colors', 'keep_colors', 'leiden', 'leiden_R_colors', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'sample_id_colors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'dist

In [2]:
celltypes = list(adata.obs['celltype'].cat.categories)

In [3]:
from tqdm import tqdm
degs = {}
for c in tqdm(celltypes):
    tmp = adata[adata.obs['celltype'] == c].copy()
    try:
        sc.tl.rank_genes_groups(tmp, groupby = 'condition', method = 'wilcoxon', pts = True)
        degs[c] = tools.returnDEres(tmp, column = 'VSV')
    except:
        pass

  0%|          | 0/27 [00:00<?, ?it/s]/home/jovyan/scripts/kttools/tools/sc/_sc.py:143: FutureWarning: Passing 'suffixes' which cause duplicate columns {'0_x'} in the result is deprecated and will raise a MergeError in a future version.
  lambda left, right: pd.merge(left, right, left_index=True, right_index=True),
100%|██████████| 27/27 [00:19<00:00,  1.41it/s]


In [4]:
degs['Tfh']

,scores,logfoldchanges,pvals,pvals_adj,pts_VSV,pts_rest
Ifi27l2a,6.580109,2.681389,4.701048e-11,5.095936e-07,0.993151,0.766667
Cmss1,6.243726,4.990635,4.272672e-10,3.087718e-06,0.897260,0.200000
Stat5b,5.948654,3.365290,2.703561e-09,1.172264e-05,0.808219,0.266667
Nme2,5.889640,5.653367,3.870383e-09,1.398498e-05,0.705479,0.066667
Camk4,5.334904,2.588426,9.559502e-08,1.884091e-04,0.863014,0.400000
...,...,...,...,...,...,...
Dusp1,-4.719187,-2.032135,2.367891e-06,2.201913e-03,0.472603,0.900000
Junb,-4.925737,-0.897313,8.404276e-07,1.138779e-03,0.986301,0.966667
Uba52,-5.252284,-1.992869,1.502246e-07,2.505284e-04,0.506849,0.833333
Ier5l,-5.447031,-3.396755,5.121737e-08,1.110393e-04,0.198630,0.766667


In [5]:
t_g = degs['Tfh']
t_g[(t_g.pts_VSV >= .25) & (t_g.pvals_adj < .05) & (t_g.logfoldchanges > 1)]

,scores,logfoldchanges,pvals,pvals_adj,pts_VSV,pts_rest
Ifi27l2a,6.580109,2.681389,4.701048e-11,5.095936e-07,0.993151,0.766667
Cmss1,6.243726,4.990635,4.272672e-10,3.087718e-06,0.897260,0.200000
Stat5b,5.948654,3.365290,2.703561e-09,1.172264e-05,0.808219,0.266667
Nme2,5.889640,5.653367,3.870383e-09,1.398498e-05,0.705479,0.066667
Camk4,5.334904,2.588426,9.559502e-08,1.884091e-04,0.863014,0.400000
Atxn1,5.258185,2.198983,1.454838e-07,2.505284e-04,0.897260,0.566667
Aff3,5.008358,3.025277,5.489644e-07,7.934366e-04,0.719178,0.200000
Gm42418,4.910000,1.133814,9.107625e-07,1.161490e-03,1.000000,1.000000
Ndfip1,4.890329,1.783024,1.006677e-06,1.212486e-03,0.904110,0.666667
Gm26740,4.849019,1.992880,1.240737e-06,1.415746e-03,0.938356,0.633333


In [6]:
list(t_g[(t_g.pts_VSV >= .25) & (t_g.pvals_adj < .05) & (t_g.logfoldchanges > 1)].index)

['Ifi27l2a',
 'Cmss1',
 'Stat5b',
 'Nme2',
 'Camk4',
 'Atxn1',
 'Aff3',
 'Gm42418',
 'Ndfip1',
 'Gm26740',
 'Gm20275',
 'Ncoa2',
 'Gmfg',
 'Gramd3',
 'Emb',
 'Camk1d',
 'Sik3',
 'Thada',
 'Srsf7',
 'Atp5o',
 'Cblb',
 'Elmo1',
 'Rnf157',
 'Dock2',
 'Lrba',
 'Cdk8',
 'Skap1',
 'Satb1',
 'Raet1e',
 'Fam169b',
 'Grcc10',
 'Dip2b',
 'Inpp4b',
 'Gphn',
 'Atp6v0c',
 'Fryl',
 'Epsti1',
 'Bzw2',
 'Cox4i1',
 'Kdm2b',
 'Timm23',
 'Adk',
 'Btbd9',
 'Rab43',
 'Bach2']